# Task 1
## 2

Next, take the two well-known datasets: Fashion MNIST (introduced in Ch 10, p.  298) and CIFAR-10.The first dataset contains 2D (grayscale) images of size 28x28, split into 10 categories; 60,000 images for training  and  10,000  for  testing,  while  the  latter  contains  32x32x3  RGB  images  (50,000/10,000train/test). Apply two reference networks on the fashion MNIST dataset

### b
A convolutional neural network

In [2]:
# Importing libraries
import tensorflow as tf
from tensorflow import keras
import sklearn
from sklearn.datasets import load_sample_image
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# Loading the data
fashion_mnist = keras.datasets.fashion_mnist

In [4]:
# Creating the train and test sets
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

# Check the dimensions 
batch_size, height, width = X_train_full.shape
print(batch_size, height, width)
input_shape = (height, width, 1) #because 1 channel

# As it only has 1 channel we want to reshape it for symmetry
X_train_full = X_train_full.reshape(batch_size, height, width, 1)
X_test = X_test.reshape(X_test.shape[0], height, width, 1)

# Normalising the pixel values to the range [0, 1]
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0

# Creating a validation set of 10%
validation_size = int(0.1 * len(X_train_full))

X_valid, X_train = X_train_full[:validation_size], X_train_full[validation_size:]
y_valid, y_train = y_train_full[:validation_size], y_train_full[validation_size:]

60000 28 28


In [5]:
# For more efficient code we"ll change the data types to float32
X_train = X_train.astype('float32')
X_valid = X_valid.astype('float32')
X_test = X_test.astype('float32')

In [26]:
# Lets first try the model from the book
model = keras.models.Sequential([
    keras.layers.Conv2D(64, 7, activation="relu", padding="same",
                        input_shape = input_shape), # No stride because images are not very large
    keras.layers.MaxPooling2D(2), # divides each spatial dimension by 2
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"), # double filters
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2), # divide dimensions by 2
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"), # again double filters
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2), 
    # fully connected network:
    keras.layers.Flatten(), # needs to be 1D for dense network
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5), # droupout of 50% to prevent overfitting
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation="softmax") # output layer
])

C:\Users\hsuik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [34]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 64)     │         3,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 14, 14, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 14, 14, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 7, 7, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 7, 7, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 3, 3, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,413,836 (5.39 MB)

 Trainable params: 1,413,834 (5.39 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [28]:
model.compile(loss = "sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

Let's train the model on the data

In [30]:
model.fit(X_train, y_train, batch_size=64, epochs=12, validation_data = (X_valid, y_valid))

Epoch 1/12
844/844 ━━━━━━━━━━━━━━━━━━━━ 153s 181ms/step - accuracy: 0.6105 - loss: 1.0420 - val_accuracy: 0.7513 - val_loss: 0.6913
Epoch 2/12
844/844 ━━━━━━━━━━━━━━━━━━━━ 164s 195ms/step - accuracy: 0.6803 - loss: 0.8617 - val_accuracy: 0.7772 - val_loss: 0.5991
Epoch 3/12
844/844 ━━━━━━━━━━━━━━━━━━━━ 205s 243ms/step - accuracy: 0.7248 - loss: 0.7464 - val_accuracy: 0.8045 - val_loss: 0.5210
Epoch 4/12
844/844 ━━━━━━━━━━━━━━━━━━━━ 205s 243ms/step - accuracy: 0.7537 - loss: 0.6771 - val_accuracy: 0.8205 - val_loss: 0.4922
Epoch 5/12
844/844 ━━━━━━━━━━━━━━━━━━━━ 225s 267ms/step - accuracy: 0.7758 - loss: 0.6168 - val_accuracy: 0.8308 - val_loss: 0.4667
Epoch 6/12
844/844 ━━━━━━━━━━━━━━━━━━━━ 256s 304ms/step - accuracy: 0.7914 - loss: 0.5843 - val_accuracy: 0.8427 - val_loss: 0.4333
Epoch 7/12
844/844 ━━━━━━━━━━━━━━━━━━━━ 152s 181ms/step - accuracy: 0.8012 - loss: 0.5542 - val_accuracy: 0.8408 - val_loss: 0.4258
Epoch 8/12
844/844 ━━━━━━━━━━━━━━━━━━━━ 176s 208ms/step - accuracy: 0.8134 -

I put a break in between for the health of my laptop. If needed ill keep adding epochs in steps of 2

In [32]:
# lets try 2 more. 14 in total now
model.fit(X_train, y_train, batch_size=64, epochs=2, validation_data = (X_valid, y_valid))

Epoch 1/2
844/844 ━━━━━━━━━━━━━━━━━━━━ 143s 169ms/step - accuracy: 0.8419 - loss: 0.4554 - val_accuracy: 0.8705 - val_loss: 0.3512
Epoch 2/2
844/844 ━━━━━━━━━━━━━━━━━━━━ 192s 228ms/step - accuracy: 0.8487 - loss: 0.4360 - val_accuracy: 0.8732 - val_loss: 0.3538


In [35]:
# 2 more, now 16
model.fit(X_train, y_train, batch_size=64, epochs=2, validation_data = (X_valid, y_valid))

Epoch 1/2
844/844 ━━━━━━━━━━━━━━━━━━━━ 134s 158ms/step - accuracy: 0.8482 - loss: 0.4289 - val_accuracy: 0.8735 - val_loss: 0.3348
Epoch 2/2
844/844 ━━━━━━━━━━━━━━━━━━━━ 160s 189ms/step - accuracy: 0.8494 - loss: 0.4259 - val_accuracy: 0.8797 - val_loss: 0.3319


In [37]:
# 4 more, now 20
model.fit(X_train, y_train, batch_size=64, epochs=4, validation_data = (X_valid, y_valid))

Epoch 1/4
844/844 ━━━━━━━━━━━━━━━━━━━━ 141s 168ms/step - accuracy: 0.8559 - loss: 0.4202 - val_accuracy: 0.8748 - val_loss: 0.3282
Epoch 2/4
844/844 ━━━━━━━━━━━━━━━━━━━━ 173s 205ms/step - accuracy: 0.8618 - loss: 0.4018 - val_accuracy: 0.8818 - val_loss: 0.3296
Epoch 3/4
844/844 ━━━━━━━━━━━━━━━━━━━━ 207s 245ms/step - accuracy: 0.8618 - loss: 0.3985 - val_accuracy: 0.8762 - val_loss: 0.3352
Epoch 4/4
844/844 ━━━━━━━━━━━━━━━━━━━━ 207s 245ms/step - accuracy: 0.8658 - loss: 0.3851 - val_accuracy: 0.8853 - val_loss: 0.3154


In [38]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.345167875289917
Test accuracy: 0.8741000294685364


This took quite a while. In theory this could give an accuracy of 92% on the test set according to the book. This is already better than the results from the MLP model. The official repository mentioned in the assignment contained a file called example_mnist_cnn.py. They made a different cnn model which they ran on the MNIST dataset. They claim to score over 99% accuracy on the test set in only 12 epochs. I used this model with some modifications in the following section to see how it would perform on the fashion_MNIST set.

In [39]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_valid = keras.utils.to_categorical(y_valid, 10)
y_test = keras.utils.to_categorical(y_test, 10)

model_cnn = keras.models.Sequential([
        keras.layers.Conv2D(32, kernel_size=(3,3),
                                activation= 'relu',
                                input_shape=input_shape), #uses a smaller filter size. No padding function is suplied so the default "valid"
                                                        #will be used. This means a reduction in dimension will be applied
        keras.layers.Conv2D(64, (3,3), activation='relu'), #uses less convolutional layers and of smaller size
        keras.layers.MaxPooling2D(pool_size=(2, 2)), # only one time maxpooling used
        keras.layers.Dropout(0.25), # smaller dropout
        # fully connected network:
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(10, activation='softmax') # output layer
])


model_cnn.compile(loss=keras.losses.categorical_crossentropy, # because output is changed to categorical instead of integers
                optimizer=keras.optimizers.Adadelta(), # uses a different optimizer. Adadelta has an adaptive learning rate so no need for manual tweeking
                metrics=['accuracy'])

model_cnn.fit(X_train, y_train,
        batch_size=128,
        epochs=12,
        verbose=1,
        validation_data=(X_valid, y_valid))

C:\Users\hsuik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/12
422/422 ━━━━━━━━━━━━━━━━━━━━ 33s 75ms/step - accuracy: 0.1058 - loss: 2.3027 - val_accuracy: 0.3632 - val_loss: 2.2230
Epoch 2/12
422/422 ━━━━━━━━━━━━━━━━━━━━ 31s 75ms/step - accuracy: 0.2311 - loss: 2.2150 - val_accuracy: 0.5563 - val_loss: 2.1150
Epoch 3/12
422/422 ━━━━━━━━━━━━━━━━━━━━ 34s 80ms/step - accuracy: 0.3233 - loss: 2.1149 - val_accuracy: 0.5910 - val_loss: 1.9829
Epoch 4/12
422/422 ━━━━━━━━━━━━━━━━━━━━ 35s 83ms/step - accuracy: 0.3890 - loss: 1.9941 - val_accuracy: 0.6025 - val_loss: 1.8298
Epoch 5/12
422/422 ━━━━━━━━━━━━━━━━━━━━ 37s 87ms/step - accuracy: 0.4430 - loss: 1.8561 - val_accuracy: 0.6180 - val_loss: 1.6637
Epoch 6/12
422/422 ━━━━━━━━━━━━━━━━━━━━ 39s 92ms/step - accuracy: 0.4689 - loss: 1.7144 - val_accuracy: 0.6327 - val_loss: 1.5022
Epoch 7/12
422/422 ━━━━━━━━━━━━━━━━━━━━ 36s 84ms/step - accuracy: 0.4961 - loss: 1.5834 - val_accuracy: 0.6418 - val_loss: 1.3608
Epoch 8/12
422/422 ━━━━━━━━━━━━━━━━━━━━ 38s 89ms/step - accuracy: 0.5179 - loss: 1.4753 - 

In [41]:
score = model_cnn.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.010797142982483
Test accuracy: 0.6638000011444092


This performs worse on the data than the example from the book but we can use it to modify things

Now lets try to edit some things to see if it becomes more efficient. Ideas:
- add an extra convolutional layer 
- keep output categories as integers as this won't require convertion
- decrease dropout last time
- keep adadelta for the learnig rate
- change relu to LeakyReLU
- changing the padding

In [ ]:
# undo the transformation
y_train = np.argmax(y_train, axis=1)
y_test = np.argmax(y_test, axis= 1)
y_valid = np.argmax(y_valid, axis=1)

In [48]:
model_imp = keras.models.Sequential([
        keras.layers.Conv2D(64, 7, padding="same",
                        input_shape = input_shape), # No stride because images are not very large
        keras.layers.LeakyReLU(alpha = 0.1), # add leakyrelu
        keras.layers.MaxPooling2D(2), 
        keras.layers.Conv2D(128, 3, padding="same"),
        keras.layers.LeakyReLU(alpha = 0.1), 
        keras.layers.Conv2D(128, 3, padding="same"),
        keras.layers.LeakyReLU(alpha = 0.1),
        keras.layers.MaxPooling2D(2), # 
        keras.layers.Conv2D(256, 3, padding="same"), 
        keras.layers.LeakyReLU(alpha = 0.1),
        keras.layers.Conv2D(256, 3, padding="same"),
        keras.layers.LeakyReLU(alpha = 0.1),
        keras.layers.MaxPooling2D(2), 
        # fully connected network:
        keras.layers.Flatten(), 
        keras.layers.Dense(128),
        keras.layers.LeakyReLU(alpha = 0.1),
        keras.layers.Dropout(0.5), # droupout of 50% to prevent overfitting
        keras.layers.Dense(64),
        keras.layers.LeakyReLU(alpha = 0.1),
        keras.layers.Dropout(0.25), # change to 25%
        keras.layers.Dense(10, activation="softmax") # output layer
])


model_imp.compile(loss="sparse_categorical_crossentropy", 
                optimizer=keras.optimizers.Adadelta(), # keeep adadelta
                metrics=['accuracy'])

model_imp.fit(X_train, y_train, batch_size=64, epochs=3, validation_data=(X_valid, y_valid)) # only 3 to see if it performs similar in the beginning to previous models


Epoch 1/3
844/844 ━━━━━━━━━━━━━━━━━━━━ 153s 179ms/step - accuracy: 0.1101 - loss: 2.3013 - val_accuracy: 0.3533 - val_loss: 2.2884
Epoch 2/3
844/844 ━━━━━━━━━━━━━━━━━━━━ 194s 230ms/step - accuracy: 0.1796 - loss: 2.2862 - val_accuracy: 0.4517 - val_loss: 2.2607
Epoch 3/3
844/844 ━━━━━━━━━━━━━━━━━━━━ 173s 205ms/step - accuracy: 0.2242 - loss: 2.2548 - val_accuracy: 0.4117 - val_loss: 2.1789


In [49]:
score = model_imp.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 2.181572675704956
Test accuracy: 0.397599995136261


Unfortunately, this performed a lot worse. Lets edit the original code one feature at the time. I'll only do 3 epochs per test because of computation time. <br>
At 3 epochs the original model performed like this: <br>
-accuracy: 0.7248 - loss: 0.7464 - val_accuracy: 0.8045 - val_loss: 0.5210 <br>

- First changing relu to leaky relu

In [50]:
model_imp = keras.models.Sequential([
        keras.layers.Conv2D(64, 7, padding="same",
                        input_shape = input_shape), # No stride because images are not very large
        keras.layers.LeakyReLU(alpha = 0.1), # add leakyrelu
        keras.layers.MaxPooling2D(2), 
        keras.layers.Conv2D(128, 3, padding="same"),
        keras.layers.LeakyReLU(alpha = 0.1), 
        keras.layers.Conv2D(128, 3, padding="same"),
        keras.layers.LeakyReLU(alpha = 0.1),
        keras.layers.MaxPooling2D(2), # 
        keras.layers.Conv2D(256, 3, padding="same"), 
        keras.layers.LeakyReLU(alpha = 0.1),
        keras.layers.Conv2D(256, 3, padding="same"),
        keras.layers.LeakyReLU(alpha = 0.1),
        keras.layers.MaxPooling2D(2), 
        # fully connected network:
        keras.layers.Flatten(), 
        keras.layers.Dense(128),
        keras.layers.LeakyReLU(alpha = 0.1),
        keras.layers.Dropout(0.5), 
        keras.layers.Dense(64),
        keras.layers.LeakyReLU(alpha = 0.1),
        keras.layers.Dropout(0.5), 
        keras.layers.Dense(10, activation="softmax") # output layer
])


model_imp.compile(loss="sparse_categorical_crossentropy", 
                optimizer="sgd", 
                metrics=['accuracy'])

model_imp.fit(X_train, y_train, batch_size=64, epochs=3, validation_data=(X_valid, y_valid)) # only 3 to see if it performs similar in the beginning to previous models


Epoch 1/3


C:\Users\hsuik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\hsuik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


844/844 ━━━━━━━━━━━━━━━━━━━━ 157s 185ms/step - accuracy: 0.2798 - loss: 1.9559 - val_accuracy: 0.7107 - val_loss: 0.8080
Epoch 2/3
844/844 ━━━━━━━━━━━━━━━━━━━━ 188s 223ms/step - accuracy: 0.6408 - loss: 0.9795 - val_accuracy: 0.7612 - val_loss: 0.6557
Epoch 3/3
844/844 ━━━━━━━━━━━━━━━━━━━━ 159s 188ms/step - accuracy: 0.7106 - loss: 0.7956 - val_accuracy: 0.7803 - val_loss: 0.5784


It seems to perform slightly worse

- Decreasing the last dropout percentage to 25%

In [51]:
model_imp = keras.models.Sequential([
    keras.layers.Conv2D(64, 7, activation="relu", padding="same",
                        input_shape = input_shape), # No stride because images are not very large
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"), 
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2), 
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"), 
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2), 
    # fully connected network:
    keras.layers.Flatten(), 
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5), 
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.25), # decrease dropout
    keras.layers.Dense(10, activation="softmax") # output layer
])

model_imp.compile(loss="sparse_categorical_crossentropy", 
                optimizer="sgd", 
                metrics=['accuracy'])

model_imp.fit(X_train, y_train, batch_size=64, epochs=3, validation_data=(X_valid, y_valid)) # only 3 to see if it performs similar in the beginning to previous models

Epoch 1/3


C:\Users\hsuik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


844/844 ━━━━━━━━━━━━━━━━━━━━ 161s 190ms/step - accuracy: 0.2957 - loss: 1.9203 - val_accuracy: 0.7237 - val_loss: 0.7599
Epoch 2/3
844/844 ━━━━━━━━━━━━━━━━━━━━ 168s 199ms/step - accuracy: 0.6655 - loss: 0.9033 - val_accuracy: 0.7713 - val_loss: 0.6273
Epoch 3/3
844/844 ━━━━━━━━━━━━━━━━━━━━ 144s 171ms/step - accuracy: 0.7236 - loss: 0.7438 - val_accuracy: 0.7863 - val_loss: 0.5517


This performs similar to the original but it overfits more as the validation set performs worse. For this reason I believe that completely removing the dropout layer would result in worse results 

- Using adadelta

In [53]:
model_imp = keras.models.Sequential([
    keras.layers.Conv2D(64, 7, activation="relu", padding="same",
                        input_shape = input_shape), # No stride because images are not very large
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"), 
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2), 
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"), 
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2), 
    # fully connected network:
    keras.layers.Flatten(), 
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5), 
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.5), # decrease dropout
    keras.layers.Dense(10, activation="softmax") # output layer
])

model_imp.compile(loss="sparse_categorical_crossentropy", 
                optimizer=keras.optimizers.Adadelta(), 
                metrics=['accuracy'])

model_imp.fit(X_train, y_train, batch_size=64, epochs=3, validation_data=(X_valid, y_valid)) # only 3 to see if it performs similar in the beginning to previous models

Epoch 1/3
844/844 ━━━━━━━━━━━━━━━━━━━━ 158s 185ms/step - accuracy: 0.1015 - loss: 2.3020 - val_accuracy: 0.0990 - val_loss: 2.2942
Epoch 2/3
844/844 ━━━━━━━━━━━━━━━━━━━━ 169s 200ms/step - accuracy: 0.1324 - loss: 2.2937 - val_accuracy: 0.1380 - val_loss: 2.2814
Epoch 3/3
844/844 ━━━━━━━━━━━━━━━━━━━━ 168s 199ms/step - accuracy: 0.1552 - loss: 2.2811 - val_accuracy: 0.1523 - val_loss: 2.2589


That is a lot worse

- What if we add another convolutional layer

In [6]:
model_imp = keras.models.Sequential([
    keras.layers.Conv2D(64, 7, activation="relu", padding="same",
                        input_shape = input_shape), # No stride because images are not very large
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"), 
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2), 
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"), 
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
    keras.layers.Conv2D(256, 3, activation='relu', padding= "same"), # additional layer
    keras.layers.MaxPooling2D(2), 
    # fully connected network:
    keras.layers.Flatten(), 
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5), 
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.5), 
    keras.layers.Dense(10, activation="softmax") # output layer
])

model_imp.compile(loss="sparse_categorical_crossentropy", 
                optimizer="sgd", 
                metrics=['accuracy'])

model_imp.fit(X_train, y_train, batch_size=64, epochs=3, validation_data=(X_valid, y_valid)) # only 3 to see if it performs similar in the beginning to previous models

C:\Users\hsuik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/3
844/844 ━━━━━━━━━━━━━━━━━━━━ 247s 291ms/step - accuracy: 0.2202 - loss: 2.1341 - val_accuracy: 0.6547 - val_loss: 0.9005
Epoch 2/3
844/844 ━━━━━━━━━━━━━━━━━━━━ 291s 345ms/step - accuracy: 0.5881 - loss: 1.1034 - val_accuracy: 0.7435 - val_loss: 0.7300
Epoch 3/3
844/844 ━━━━━━━━━━━━━━━━━━━━ 276s 327ms/step - accuracy: 0.6733 - loss: 0.8841 - val_accuracy: 0.7685 - val_loss: 0.6340


This is worse than the base model.

- Changing the padding

In [9]:
model_imp = keras.models.Sequential([
    keras.layers.Conv2D(64, 7, activation="relu", padding="valid",
                        input_shape = input_shape), 
    keras.layers.MaxPooling2D(2), 
    keras.layers.Conv2D(128, 3, activation="relu", padding="valid"),
    keras.layers.Conv2D(128, 3, activation="relu", padding="valid"), #MaxPooling layer removed as output is smaller with valid padding
    keras.layers.Conv2D(256, 3, activation="relu", padding="valid"), 
    keras.layers.Conv2D(256, 3, activation="relu", padding="valid"),
    keras.layers.MaxPooling2D(2), 
    # fully connected network:
    keras.layers.Flatten(), 
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5), 
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation="softmax") 
])

model_imp.compile(loss="sparse_categorical_crossentropy", 
                optimizer="sgd", 
                metrics=['accuracy'])

model_imp.fit(X_train, y_train, batch_size=64, epochs=3, validation_data=(X_valid, y_valid))

Epoch 1/3
844/844 ━━━━━━━━━━━━━━━━━━━━ 56s 65ms/step - accuracy: 0.2286 - loss: 2.1030 - val_accuracy: 0.6370 - val_loss: 1.0333
Epoch 2/3
844/844 ━━━━━━━━━━━━━━━━━━━━ 84s 99ms/step - accuracy: 0.5554 - loss: 1.1894 - val_accuracy: 0.7077 - val_loss: 0.8173
Epoch 3/3
844/844 ━━━━━━━━━━━━━━━━━━━━ 59s 70ms/step - accuracy: 0.6590 - loss: 0.9221 - val_accuracy: 0.7538 - val_loss: 0.6606


Again this is worse than the base level.

Unfortunately non of the modifications performed better than the base model.